# My first Flask tutorial :) 

EXAMPLE SCRIPT 1

In [6]:
%%writefile myflask_HelloWorld.py
from flask import Flask

app = Flask(__name__)

@app.route('/') # decorator
def hello():
    '''viewing function'''
    return "Hello World! :)"; 

if __name__ == "__main__":
    app.run(debug = True, host = "0.0.0.0", port = 3000)

Writing myflask_HelloWorld.py


Explanation

* `route` binds a given `URL` (here `/` -  the default route) to a view function (here `hello()`)

EXAMPLE SCRIPT 2

In [8]:
%%writefile myflask_TwoRoutes.py
from flask import Flask

app = Flask(__name__)

@app.route('/') # decorator
def home():
    '''viewing function - home or first landing page'''
    return "Welcome home! :)"; 


@app.route('/educative')
def leanr():
    '''viewing function - appended page'''
    return "Ready to learn!"

if __name__ == "__main__":
    app.run(debug = True, host = "0.0.0.0", port = 3001)

Overwriting myflask_TwoRoutes.py


SOURCE

* https://www.educative.io/blog/python-flask-tutorial 

Execute script 

In [2]:
%run myflask_HelloWorld.py

 * Serving Flask app "myflask_HelloWorld" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://0.0.0.0:3000/ (Press CTRL+C to quit)
 * Restarting with inotify reloader


# Flask-tutorial project

## app setup

ANNOTATED

In [12]:
%%writefile flask-tutorial/flaskr/__init__.py
import os

from flask import Flask

def create_app(test_config=None): # NOTE: create_app is the application factory function
    # create and configure the app
    app = Flask(__name__, instance_relative_config=True)
    # BREAKDOWN:
    # __name__ is the name of the current module: __init__ .Provides the module's path too
    # instance folder is outside flaskr, config files relative to instance folder if True
    
    app.config.from_mapping( # NOTE: set up configuration settings
        SECRET_KEY='dev', 
        DATABASE=os.path.join(app.instance_path, 'flaskr.sqlite'), 
    )
    # BREAKDOWN:
    # SECRET_KEY used to keep data safe, for deployment 'dev' should be replaced with a random key
    # DATABASE is the SQLite DB pathway
    
    if test_config is None:
        # load the instance config, if it exists, when not testing
        app.config.from_pyfile('config.py', silent=True)
        
        # NOTE: takes config settings from 'config.py' (if exists) in instance folder (here: flask-tutorial/)    else:
        # this can also be used to store real 'SECRET_KEY'
        # load the test config if passed in
        app.config.from_mapping(test_config)

    # ensure the instance folder exists
    try:
        os.makedirs(app.instance_path)
        # NOTE: Flask doesn't create instance folder by default. Needs manual creation
    except OSError:
        pass

    # a simple page that says hello
    @app.route('/')
    # NOTE: create a simple route by binding an URL to a view function
    def hello():
        '''view function'''
        return 'Hello, World!'

    return app

Overwriting flask-tutorial/flaskr/__init__.py


ORIGINAL

__TEST__ Execution works fine for original, but not for annotated version. 

In [13]:
%%writefile flask-tutorial/flaskr/__init__.py
import os

from flask import Flask

def create_app(test_config=None):
    # create and configure the app
    app = Flask(__name__, instance_relative_config=True)
    app.config.from_mapping(
        SECRET_KEY='dev',
        DATABASE=os.path.join(app.instance_path, 'flaskr.sqlite'),
    )

    if test_config is None:
        # load the instance config, if it exists, when not testing
        app.config.from_pyfile('config.py', silent=True)
    else:
        # load the test config if passed in
        app.config.from_mapping(test_config)

    # ensure the instance folder exists
    try:
        os.makedirs(app.instance_path)
    except OSError:
        pass

    # a simple page that says hello
    @app.route('/')
    def hello():
        return 'Hello, World!'

    return app

Overwriting flask-tutorial/flaskr/__init__.py


## Terminal execution

```bash
path=/data/dragon000/sanjuan/research/software_hands-on/webdevel/flask-tutorial
cd $path
export FLASK_APP=flaskr
export FLASK_ENV=development
flask run
```

## SQLite database

### `dp.py` 

ANNOTATED

In [5]:
%%writefile flask-tutorial/flaskr/db.py
import sqlite3

import click
from flask import current_app, g # special objects
from flask.cli import with_appcontext


def get_db():
    if 'db' not in g:
        g.db = sqlite3.connect(
            current_app.config['DATABASE'],
            detect_types=sqlite3.PARSE_DECLTYPES
        )
        g.db.row_factory = sqlite3.Row
        # BREAKDOWN
        # Flask app handling request (current_app)
        # sqlite3.connect() establishes connection to DB file
        # DATABASE default keyword
    return g.db

def close_db(e=None):
    db = g.pop('db', None)

    if db is not None:
        db.close()
        
def init_db():
    db = get_db()
    # take file with sqilte raw expressions
    with current_app.open_resource('schema.sql') as f:
        db.executescript(f.read().decode('utf8'))

@click.command('init-db')
@with_appcontext
def init_db_command():
    """Clear the existing data and create new tables."""
    init_db()
    click.echo('Initialized the database.')
    
# Register close_db() and init_db_command() to app
def init_app(app):
    app.teardown_appcontext(close_db)
    app.cli.add_command(init_db_command)

Overwriting flask-tutorial/flaskr/db.py


### `schema.sql`

SQLite raw expressions

* Generates two tables: `user` and `post`
* Columns are specified as `id`, `username`, `password`, with their correspionding data ttraibutes.

In [6]:
%%writefile flask-tutorial/flaskr/schema.sql
DROP TABLE IF EXISTS user;
DROP TABLE IF EXISTS post;

CREATE TABLE user (
  id INTEGER PRIMARY KEY AUTOINCREMENT,
  username TEXT UNIQUE NOT NULL,
  password TEXT NOT NULL
);

CREATE TABLE post (
  id INTEGER PRIMARY KEY AUTOINCREMENT,
  author_id INTEGER NOT NULL,
  created TIMESTAMP NOT NULL DEFAULT CURRENT_TIMESTAMP,
  title TEXT NOT NULL,
  body TEXT NOT NULL,
  FOREIGN KEY (author_id) REFERENCES user (id)
);

Overwriting flask-tutorial/flaskr/schema.sql


### modfied `__init__.py`

In [7]:
%%writefile flask-tutorial/flaskr/__init__.py
import os

from flask import Flask

def create_app(test_config=None):
    # create and configure the app
    app = Flask(__name__, instance_relative_config=True)
    app.config.from_mapping(
        SECRET_KEY='dev',
        DATABASE=os.path.join(app.instance_path, 'flaskr.sqlite'),
    )

    if test_config is None:
        # load the instance config, if it exists, when not testing
        app.config.from_pyfile('config.py', silent=True)
    else:
        # load the test config if passed in
        app.config.from_mapping(test_config)

    # ensure the instance folder exists
    try:
        os.makedirs(app.instance_path)
    except OSError:
        pass

    # a simple page that says hello
    @app.route('/')
    def hello():
        return 'Hello, World!'

    from . import db
    db.init_app(app)
    
    return app

Overwriting flask-tutorial/flaskr/__init__.py
